## This follows a CNN approach as follows

1. Convolve over each image using a filter
2. This gives us a feature map
3. At the end, use a classification head and a binary loss function

### Imports

In [1]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image

from dataset_loader import TRAIN_SEGMENTATION_DATASET, TEST_SEGMENTATION_DATASET
from cnn_model import CNN_MODEL
from cnn_config import cnn_experiment_1

from trainer_evaluator import TRAINER_EVALUATOR

### Data Loader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor()
])

# Create dataset
train_dataset = TRAIN_SEGMENTATION_DATASET(
    "/Users/tejasdhopavkar/Documents/MS/Saarland_University/Semester_1/ML/Project/dataset/training/images",
    "/Users/tejasdhopavkar/Documents/MS/Saarland_University/Semester_1/ML/Project/dataset/training/ground_truth",
    transform
)

test_dataset = TEST_SEGMENTATION_DATASET(
    "/Users/tejasdhopavkar/Documents/MS/Saarland_University/Semester_1/ML/Project/dataset/test/images",
    transform
)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

# Example batch
images, labels, filename = next(iter(train_loader))
print(images.shape, labels.shape, filename)

torch.Size([1, 3, 375, 500]) torch.Size([1, 1, 375, 500]) ('2009_002082.jpg',)


### Model, criterion, optimizer

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else
                      "mps" if torch.backends.mps.is_available() else "cpu")

In [4]:
model = CNN_MODEL(cnn_experiment_1['model_args']['input_size'], cnn_experiment_1['model_args']['hidden_layers'], cnn_experiment_1['model_args']['activation'], cnn_experiment_1['model_args']['norm_layer'], cnn_experiment_1['model_args']['drop_prob'], cnn_experiment_1['model_args']['max_pooling']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss = nn.BCELoss()

In [5]:
model

CNN_MODEL(
  (features): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Conv2d(512, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(128, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
  )
)

### Model training

In [6]:
trainer_evaluator = TRAINER_EVALUATOR()

In [7]:
print("Training started...")
trainer_evaluator.train_cnn(model, train_loader, optimizer, loss, device, 25)

Training started...
Epoch 0: Started training
Total loss after epoch 0: 695.8074588477612
Epoch 1/25 - Loss: 3.0518
Epoch 1: Started training
Total loss after epoch 1: 695.80745883286
Epoch 2/25 - Loss: 3.0518
Epoch 2: Started training
Total loss after epoch 2: 695.80745883286
Epoch 3/25 - Loss: 3.0518
Epoch 3: Started training
Total loss after epoch 3: 695.80745883286
Epoch 4/25 - Loss: 3.0518
Epoch 4: Started training
Total loss after epoch 4: 695.80745883286
Epoch 5/25 - Loss: 3.0518
Epoch 5: Started training
Total loss after epoch 5: 695.80745883286
Epoch 6/25 - Loss: 3.0518
Epoch 6: Started training
Total loss after epoch 6: 695.80745883286
Epoch 7/25 - Loss: 3.0518
Epoch 7: Started training
Total loss after epoch 7: 695.80745883286
Epoch 8/25 - Loss: 3.0518
Epoch 8: Started training
Total loss after epoch 8: 695.80745883286
Epoch 9/25 - Loss: 3.0518
Epoch 9: Started training
Total loss after epoch 9: 695.80745883286
Epoch 10/25 - Loss: 3.0518
Epoch 10: Started training
Total loss

### Save predictions

In [7]:
# Save predicted masks
trainer_evaluator.save_predictions(model, train_loader, device)

/Users/tejasdhopavkar/Documents/MS/Saarland_University/Semester_1/ML/Project/trainer_evaluator.py:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(to

KeyboardInterrupt: 